In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('collected_data1.csv')

C:\Users\Dushyant\AppData\Local\Temp\ipykernel_1776\1079090705.py:1: DtypeWarning: Columns (17,18,51,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('collected_data1.csv')


In [3]:
df.shape

(179549, 60)

In [4]:
df.head()

,jobma_catcher_id,credit_amount,wallet_amount,subscription_type_x,plan_type,is_unlimited,premium_storage_x,premium_plan,subscription_amount,credit_given,...,jobma_job_type,jobma_job_functional_areas,jobma_job_keywords,jobma_job_currency,jobma_job_company_profile,jobma_job_apply_count,jobma_job_view_count,approval,job_subscription_status,applicant_count
0,92,NaN,34.0,No,No,No,0,"{""subscription_charges"": ""10000"", ""pre_recorde...",0.0,0,...,Permanent,HR / Administration / IR,"java, python",NaN,NaN,NaN,NaN,1.0,1,NaN
1,92,NaN,34.0,No,No,No,0,"{""subscription_charges"": ""10000"", ""pre_recorde...",0.0,0,...,Permanent,HR / Administration / IR,"Training, ""Instructional Design"", ""Curriculum...",NaN,NaN,NaN,NaN,1.0,1,NaN
2,92,NaN,34.0,No,No,No,0,"{""subscription_charges"": ""10000"", ""pre_recorde...",0.0,0,...,Permanent,Customer Service,"""International Sales"",""Lead Generation"",""Inter...",NaN,NaN,NaN,NaN,1.0,1,NaN
3,92,NaN,34.0,No,No,No,0,"{""subscription_charges"": ""10000"", ""pre_recorde...",0.0,0,...,Permanent,Hotels / Restaurants,java,NaN,NaN,NaN,NaN,1.0,1,NaN
4,92,NaN,34.0,No,No,No,0,"{""subscription_charges"": ""10000"", ""pre_recorde...",0.0,0,...,Permanent,Customer Service,"Team Leader, java ,developer",NaN,NaN,NaN,NaN,1.0,1,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179549 entries, 0 to 179548
Data columns (total 60 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   jobma_catcher_id                179549 non-null  int64  
 1   credit_amount                   143663 non-null  float64
 2   wallet_amount                   179549 non-null  float64
 3   subscription_type_x             179549 non-null  object 
 4   plan_type                       179545 non-null  object 
 5   is_unlimited                    179546 non-null  object 
 6   premium_storage_x               179549 non-null  int64  
 7   premium_plan                    179521 non-null  object 
 8   subscription_amount             179549 non-null  float64
 9   credit_given                    179549 non-null  int64  
 10  premium_storage_y               179549 non-null  int64  
 11  currency_val                    179549 non-null  float64
 12  payment_mode    

In [6]:
df.isnull().sum()

jobma_catcher_id                       0
credit_amount                      35886
wallet_amount                          0
subscription_type_x                    0
plan_type                              4
is_unlimited                           3
premium_storage_x                      0
premium_plan                          28
subscription_amount                    0
credit_given                           0
premium_storage_y                      0
currency_val                           0
payment_mode                          29
status                                 0
payment_status                       278
number_of_transactions                 0
jobma_catcher_company                  0
org_type                          175794
jobma_catcher_indus                 9814
jobma_catcher_title                    0
jobma_catcher_otype                 6773
jobma_catcher_creation                 0
jobma_catcher_type                     0
jobma_catcher_sub_accounts        179549
is_premium      